In [ ]:
# Install dependencies
!pip install -q gradio sentence-transformers groq

# Imports
import gradio as gr
import os
import re
import requests
from sentence_transformers import SentenceTransformer, util

# 🔐 Set your Groq API Key here
os.environ['GROQ_API_KEY'] = "YOUR_GROQ_API_KEY"  # Replace this with your actual Groq key

# Load sentence transformer
model = SentenceTransformer("all-MiniLM-L6-v2")

# Sample corpus for plagiarism detection
corpus = [
    "Artificial intelligence is transforming the world.",
    "Climate change is one of the greatest challenges.",
    "The mitochondria is the powerhouse of the cell.",
    "Quantum computing uses principles of quantum mechanics."
]
corpus_embeddings = model.encode(corpus, convert_to_tensor=True)

# Plagiarism check
def check_plagiarism(sentences):
    results = []
    for sent in sentences:
        query_embedding = model.encode(sent, convert_to_tensor=True)
        hits = util.semantic_search(query_embedding, corpus_embeddings, top_k=1)
        score = hits[0][0]['score']
        if score > 0.75:
            results.append(("plagiarized", sent))
        else:
            results.append(("original", sent))
    return results

# Detect AI using LLaMA 3 via Groq
def detect_ai_sentences(text):
    prompt = f"""You are an expert AI text detector. Return a list of sentence indexes (0-based) that are likely AI-generated in the given text:\n\n{text}"""

    headers = {
        "Authorization": f"Bearer {os.environ['GROQ_API_KEY']}",
        "Content-Type": "application/json"
    }
    data = {
        "model": "llama3-70b-8192",
        "messages": [{"role": "user", "content": prompt}],
        "temperature": 0.3
    }

    response = requests.post("https://api.groq.com/openai/v1/chat/completions", headers=headers, json=data)
    content = response.json()["choices"][0]["message"]["content"]

    try:
        indexes = list(map(int, re.findall(r'\d+', content)))
    except:
        indexes = []
    return indexes

# Humanize text using LLaMA 3
def humanize_text(text):
    prompt = f"""Rewrite the following text to sound more natural, human-like, and less AI-generated:\n\n{text}"""

    headers = {
        "Authorization": f"Bearer {os.environ['GROQ_API_KEY']}",
        "Content-Type": "application/json"
    }
    data = {
        "model": "llama3-70b-8192",
        "messages": [{"role": "user", "content": prompt}],
        "temperature": 0.7
    }

    response = requests.post("https://api.groq.com/openai/v1/chat/completions", headers=headers, json=data)
    return response.json()["choices"][0]["message"]["content"]

# Highlight AI and plagiarism
def highlight_text(text, plagiarism_result, ai_indexes):
    sentences = re.split(r'(?<=[.!?]) +', text)
    html = ""
    for i, sent in enumerate(sentences):
        styles = []
        if plagiarism_result[i][0] == "plagiarized":
            styles.append("background-color:#ffcccc")  # red
        if i in ai_indexes:
            styles.append("background-color:#fff4cc")  # yellow
        if styles:
            html += f"<span style='{';'.join(styles)}'>{sent}</span> "
        else:
            html += sent + " "
    return html.strip()

# Final logic
def analyze_text(input_text):
    words = input_text.split()
    if len(words) > 1500:
        return "⚠️ Please limit your input to 1500 words.", "", "", ""

    sentences = re.split(r'(?<=[.!?]) +', input_text)

    # Plagiarism check
    plagiarism_result = check_plagiarism(sentences)
    plagiarized_sentences = [s for tag, s in plagiarism_result if tag == "plagiarized"]

    # AI detection
    ai_indexes = detect_ai_sentences(input_text)

    # Highlight output
    html_result = highlight_text(input_text, plagiarism_result, ai_indexes)

    # Percentages
    total = len(sentences)
    ai_percent = (len(ai_indexes) / total) * 100 if total > 0 else 0
    plagiarism_percent = (len(plagiarized_sentences) / total) * 100 if total > 0 else 0

    # Humanized text
    humanized = humanize_text(input_text)

    return (
        html_result,
        f"{ai_percent:.2f}%",
        f"{plagiarism_percent:.2f}%",
        humanized
    )

# Gradio UI
with gr.Blocks() as demo:
    gr.Markdown("## 🔍 AI & Plagiarism Detector + Humanizer")
    gr.Markdown("""
- 🔴 Red = Plagiarized text
- 🟡 Yellow = AI-generated text
- Get AI % and Plagiarism %
- Bonus: Humanized rewrite of your text
""")

    text_input = gr.Textbox(label="Input Text (max 1500 words)", lines=15, placeholder="Paste your content here...")

    analyze_button = gr.Button("Analyze")

    html_output = gr.HTML()
    ai_percent_output = gr.Textbox(label="AI-Generated Percentage", interactive=False)
    plagiarism_percent_output = gr.Textbox(label="Plagiarism Percentage", interactive=False)
    humanized_output = gr.Textbox(label="Humanized Text", lines=10)

    analyze_button.click(
        analyze_text,
        inputs=text_input,
        outputs=[
            html_output,
            ai_percent_output,
            plagiarism_percent_output,
            humanized_output
        ]
    )

demo.launch()


It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://36c4c8fb6c743116ef.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
